**Note**: needs to be run separetely in the rtv environment (not the MBM). Computes the sky view factor from DEMs (needed as inputs in the MBM).

In [ ]:
# --- Standard library ---
import os
import glob
import shutil
import warnings
import sys

sys.path.append(os.path.join(os.getcwd(), "../../"))

# --- Scientific / numeric ---
import numpy as np
import xarray as xr

# --- Geospatial ---
import rasterio
import rioxarray
import rvt.default
import rvt.vis

# --- Plotting / progress ---
import matplotlib.pyplot as plt
from tqdm import tqdm

from regions.Switzerland.scripts.svf_processing import *

# --- Misc settings ---
warnings.filterwarnings("ignore")
cfg_datapath = '/scratch-3/vmarijn/MassBalanceMachine/../data/'

In [ ]:
# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVF_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315

DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]

## RGI:

### Compute skyview factor:

In [ ]:
path_rgi_NOR = 'RGI_v6/RGI_08_Scandinavia/'

In [ ]:
# -------------------
# Paths & parameters
# -------------------
path_geotiff = os.path.join(cfg_datapath, path_rgi_NOR, "geotiff")
path_out = os.path.join(cfg_datapath, path_rgi_NOR, "svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)


# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "RGI60-08.*.tif")))
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path,
                                  out_nc,
                                  svf_n_dir=SVF_N_DIR,
                                  svf_r_max=SVF_R_MAX,
                                  svf_noise=SVF_NOISE,
                                  asvf_level=ASVF_LEVEL,
                                  asvf_dir=ASVF_DIR)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# --- Path to one of your generated SVF NetCDFs ---
nigardsbreen_rgi = "RGI60-08.01126"
path_svf = os.path.join(cfg_datapath, path_rgi_NOR, "svf_nc",
                        f"{nigardsbreen_rgi}_svf.nc")

# --- Load dataset ---
ds = xr.open_dataset(path_svf)

# get x and y bounds of the glacier
x_min, x_max = float(ds.x.min()), float(ds.x.max())
y_min, y_max = float(ds.y.min()), float(ds.y.max())

print(f"x: {x_min} to {x_max}")
print(f"y: {y_min} to {y_max}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("x (m)")
else:
    axes[1].set_visible(False)

# Positive Openness
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("x (m)")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{os.path.basename(path_svf)}", fontsize=14)
plt.show()

In [ ]:
rgi_id = "RGI60-08.01258"
path_svf = os.path.join(cfg_datapath, path_rgi_NOR, "svf_nc",
                        f"{rgi_id}_svf.nc")

# --- Load dataset ---
≈ = xr.open_dataset(path_svf)

# get x and y bounds of the glacier
x_min, x_max = float(ds.x.min()), float(ds.x.max())
y_min, y_max = float(ds.y.min()), float(ds.y.max())

print(f"x: {x_min} to {x_max}")
print(f"y: {y_min} to {y_max}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("x (m)")
else:
    axes[1].set_visible(False)

# Positive Openness
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("x (m)")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{os.path.basename(path_svf)}", fontsize=14)
plt.show()

### Reproject to Lat/Lon:

In [ ]:
# -------- paths --------
path_nc_xy = os.path.join(cfg_datapath, path_rgi_NOR,
                          "svf_nc/")  # input: x/y .nc
path_nc_ll = os.path.join(cfg_datapath, path_rgi_NOR,
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)


def main():
    files = sorted(glob.glob(os.path.join(path_nc_xy, "RGI60-08.*svf.nc")))
    print(f"Found {len(files)} x/y SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(
                f,
                out_path,
                data_vars=DATA_VARS,
                #  src_crs=SRC_CRS,
                dst_crs=DST_CRS)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# --- Path to one of your generated SVF NetCDFs ---
nigardsbreen_rgi = "RGI60-08.01126"
path_svf = os.path.join(
    cfg_datapath,
    path_rgi_NOR,
    "svf_nc_latlon",
    f"{nigardsbreen_rgi}_svf_latlon.nc",
)

ds = xr.open_dataset(path_svf)

# --- Handle coordinate names flexibly ---
if "lon" in ds.coords and "lat" in ds.coords:
    xcoord, ycoord = "lon", "lat"
elif "x" in ds.coords and "y" in ds.coords:
    # rename for clarity if needed
    ds = ds.rename({"x": "lon", "y": "lat"})
    xcoord, ycoord = "lon", "lat"
else:
    raise ValueError("Could not find lon/lat coordinates in dataset")

# Bounds
lon_min, lon_max = float(ds[xcoord].min()), float(ds[xcoord].max())
lat_min, lat_max = float(ds[ycoord].min()), float(ds[ycoord].max())

print(f"lon: {lon_min:.5f} to {lon_max:.5f}")
print(f"lat: {lat_min:.5f} to {lat_max:.5f}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], x=xcoord, y=ycoord, cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("Longitude (°)")
axes[0].set_ylabel("Latitude (°)")
axes[0].set_aspect("equal")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1],
                    x=xcoord,
                    y=ycoord,
                    cmap="viridis",
                    vmin=0,
                    vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("Longitude (°)")
    axes[1].set_aspect("equal")
else:
    axes[1].set_visible(False)

# OPNS
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], x=xcoord, y=ycoord, cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("Longitude (°)")
    axes[2].set_aspect("equal")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{nigardsbreen_rgi} — SVF products (lat/lon)", fontsize=14)
plt.show()

In [ ]:
# --- Path to one of your generated SVF NetCDFs ---
rgi_id = "RGI60-08.00005"
path_svf = os.path.join(
    cfg_datapath,
    path_rgi_NOR,
    "svf_nc_latlon",
    f"{rgi_id}_svf_latlon.nc",
)

ds = xr.open_dataset(path_svf)

# --- Handle coordinate names flexibly ---
if "lon" in ds.coords and "lat" in ds.coords:
    xcoord, ycoord = "lon", "lat"
elif "x" in ds.coords and "y" in ds.coords:
    # rename for clarity if needed
    ds = ds.rename({"x": "lon", "y": "lat"})
    xcoord, ycoord = "lon", "lat"
else:
    raise ValueError("Could not find lon/lat coordinates in dataset")

# Bounds
lon_min, lon_max = float(ds[xcoord].min()), float(ds[xcoord].max())
lat_min, lat_max = float(ds[ycoord].min()), float(ds[ycoord].max())

print(f"lon: {lon_min:.5f} to {lon_max:.5f}")
print(f"lat: {lat_min:.5f} to {lat_max:.5f}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], x=xcoord, y=ycoord, cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("Longitude (°)")
axes[0].set_ylabel("Latitude (°)")
axes[0].set_aspect("equal")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1],
                    x=xcoord,
                    y=ycoord,
                    cmap="viridis",
                    vmin=0,
                    vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("Longitude (°)")
    axes[1].set_aspect("equal")
else:
    axes[1].set_visible(False)

# OPNS
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], x=xcoord, y=ycoord, cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("Longitude (°)")
    axes[2].set_aspect("equal")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{rgi_id} — SVF products (lat/lon)", fontsize=14)
plt.show()